In [1]:
import tweepy       # python wrapper for twitter api
import json
import time
import requests
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer

# step 0, get your own twitter credentials!

In [2]:
# this is my own personal twitter api information
# if you could be so kind as to sign up yourself on both twitter and mashape that'd be great :)
# It's FREEEEEEE
api_key = 'g3xAZVinye67J9guxFhGT5Y8f'
api_secret = 'TYkqHGyBxNSoziaap8LtygafZpD6cHjLuiIdgfbvNRr3RV2fwU'
access_token = '536799553-4jpBdlg8XMtnXCVEA2daxrDYuwT4HTQNx7eEOtoJ'
access_secret = 'Mya4FnubounjlXM1Vh4fwDTLwOjIT6UbTNiBzQQUEAJqI'

In [3]:
# Documentation is your friend! http://docs.tweepy.org/en/v3.1.0/
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth) # returns a tweepy authorization handler object

In [4]:
# 1. Use Twitter API to stream and store tweets (300 per candidate)
# explain geocode is doing
bernie_tweets =  api.search(q="#berniesanders", count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#bernie', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#FeelTheBern', count=100, geocode="37.781157,-122.398720,50mi")
hilary_tweets = api.search(q='#ImWithHer', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#hillaryclinton', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#hillary2016', count=100, geocode="37.781157,-122.398720,50mi")
the_donald_tweets = api.search(q='#donaldtrump', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#trump2016', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#trump', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#makeamericagreatagain', count=100, geocode="37.781157,-122.398720,50mi")

Geocode is returning only tweets from within 50 miles of the given coordinates

In [16]:
the_donald_tweets[0]

[datetime.datetime(2016, 5, 10, 1, 17, 59),
 u'@donnabrazile why is anything #DonaldTrump has said or done negotiable with the mass #GOP? This is the moment they need to be principled.',
 0,
 False,
 u'en',
 None,
 0,
 False,
 None]

In [17]:
# wrappers come with built in python attributes and methods!
print dir(the_donald_tweets[0])

['__add__', '__class__', '__contains__', '__delattr__', '__delitem__', '__delslice__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getslice__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__setslice__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']


In [18]:
print the_donald_tweets[0].created_at
print the_donald_tweets[0].text
print the_donald_tweets[0].favorite_count
print the_donald_tweets[0].favorited
print the_donald_tweets[0].lang
print the_donald_tweets[0].geo
print the_donald_tweets[0].retweet_count
print the_donald_tweets[0].retweeted
print the_donald_tweets[0].coordinates

AttributeError: 'list' object has no attribute 'created_at'

In [9]:
def parse_status_objects(tweet_list):
    modified_tweets = list()
    for x in tweet_list:
        modified_tweets.append( [
                    x.created_at,
                    x.text,
                    x.favorite_count,
                    x.favorited,
                    x.lang,
                    x.geo,
                    x.retweet_count,
                    x.retweeted,
                    x.coordinates
                ])
    return modified_tweets
        
hilary_tweets = parse_status_objects(hilary_tweets)
the_donald_tweets = parse_status_objects(the_donald_tweets)
bernie_tweets = parse_status_objects(bernie_tweets)

In [10]:
print bernie_tweets[0]
print the_donald_tweets[0]
print hilary_tweets[0]

[datetime.datetime(2016, 5, 10, 1, 26, 42), u'RT @TerryMcSweeney: #BernieInSacramento Supporters now say some were in line since 2pm. #BernieSanders comes on 8 @nbcbayarea at 11 https:/\u2026', 0, False, u'en', None, 11, False, None]
[datetime.datetime(2016, 5, 10, 1, 17, 59), u'@donnabrazile why is anything #DonaldTrump has said or done negotiable with the mass #GOP? This is the moment they need to be principled.', 0, False, u'en', None, 0, False, None]
[datetime.datetime(2016, 5, 10, 1, 28, 22), u'RT @Mara_Shayne: Lines are out the door for our office opening here in #SanFrancisco #HillaryforCA #ImWithHer https://t.co/mdeVoLtlCn', 0, False, u'en', None, 2, False, None]


In [37]:
# 2 Create a dataframe that combines all of the tweets from each candidate
# To do so, you will need to concat the data frames
# so each row is a tweet and your columns should be 
# date, text, favorite_count,favorited, language, geocode, rwtweet count, retweeted, coordinates

Bernie = pd.DataFrame(data = bernie_tweets, columns = ['date', 'text', 'favorite_count', 'favorited', 'language', 
                                                       'geocode', 'rwtweet count', 'retweeted', 'coordinates'])

Donald = pd.DataFrame(data = the_donald_tweets, columns = ['date', 'text', 'favorite_count', 'favorited', 'language', 
                                                       'geocode', 'rwtweet count', 'retweeted', 'coordinates'])
Hilary = pd.DataFrame(data = hilary_tweets, columns = ['date', 'text', 'favorite_count', 'favorited', 'language', 
                                                       'geocode', 'rwtweet count', 'retweeted', 'coordinates'])



df = pd.concat([Bernie, Hilary, Donald], axis = 0)
pd.set_option('max_colwidth', 500)


df.iloc[-400:]

,date,text,favorite_count,favorited,language,geocode,rwtweet count,retweeted,coordinates
0,2016-05-10 01:17:59,@donnabrazile why is anything #DonaldTrump has said or done negotiable with the mass #GOP? This is the moment they need to be principled.,0,False,en,None,0,False,None
1,2016-05-10 01:11:10,"RT @sonnylebythebay: To borrow @JonHuntsman's phrase, #DonaldTrump's a well lubricated weather vane because he lacks core principles https:…",0,False,en,None,2,False,None
2,2016-05-10 01:08:20,RT @JacloPac: #DonaldTrump is a master manipulator &amp; con man who cannot be allowed to win the Presidency besides......#VoteBlue https://t.c…,0,False,en,None,120,False,None
3,2016-05-10 01:04:50,"RT @sonnylebythebay: To borrow @JonHuntsman's phrase, #DonaldTrump's a well lubricated weather vane because he lacks core principles https:…",0,False,en,None,2,False,None
4,2016-05-10 01:04:03,RT @JacloPac: #DonaldTrump is a master manipulator &amp; con man who cannot be allowed to win the Presidency besides......#VoteBlue https://t.c…,0,False,en,None,120,False,None
5,2016-05-10 01:02:00,"To borrow @JonHuntsman's phrase, #DonaldTrump's a well lubricated weather vane because he lacks core principles https://t.co/hPZSNfehCq",1,False,en,None,2,False,None
6,2016-05-10 01:01:16,RT @JacloPac: #DonaldTrump is a master manipulator &amp; con man who cannot be allowed to win the Presidency besides......#VoteBlue https://t.c…,0,False,en,None,120,False,None
7,2016-05-10 00:56:38,RT @JacloPac: #DonaldTrump is a master manipulator &amp; con man who cannot be allowed to win the Presidency besides......#VoteBlue https://t.c…,0,False,en,None,120,False,None
8,2016-05-10 00:54:11,RT @IsraelNewsNow: #IsraeltheRegion #JewishTimes #DonaldTrump #ChrisChristie #GOP Trump taps Christie to lead White House transition https:…,0,False,en,None,4,False,None
9,2016-05-10 00:52:20,"RT @sonnylebythebay: #SadiqKhan, #Muslim mayor of #London, plans US visit b4 election 'in case #DonaldTrump wins' https://t.co/d5JqF9VMgP h…",0,False,en,None,1,False,None


In [12]:
# 3 Create a function to take in a string and output the textblob sentiment of that string
def sentiment(string):
    tweet = TextBlob(string)
    return tweet.sentiment.polarity

In [13]:
# 4 Add a column to your data frame called 'sentiment' which holds the sentiment of that tweet (hint: use the function from #3)
df['sentiment'] = df.text.apply(sentiment)

In [ ]:
#df[0]?

In [15]:
# 5 create a word count column, which holds the number of words in the text column
def wordcount(string):
    tweet = TextBlob(string)
    return len(tweet.words)

df['word_count'] = df.text.apply(wordcount)
df.head(20)

,date,text,favorite_count,favorited,language,geocode,rwtweet count,retweeted,coordinates,sentiment,word_count
0,2016-05-10 01:26:42,RT @TerryMcSweeney: #BernieInSacramento Supporters now say some were in line since 2pm. #BernieSanders comes on 8 @nbcbayarea at 11 https:/…,0,False,en,None,11,False,None,0.0000,21
1,2016-05-10 01:23:32,RT @TerryMcSweeney: #BernieInSacramento Supporters now say some were in line since 2pm. #BernieSanders comes on 8 @nbcbayarea at 11 https:/…,0,False,en,None,11,False,None,0.0000,21
2,2016-05-10 01:22:53,RT @TerryMcSweeney: #BernieInSacramento Supporters now say some were in line since 2pm. #BernieSanders comes on 8 @nbcbayarea at 11 https:/…,0,False,en,None,11,False,None,0.0000,21
3,2016-05-10 01:21:53,RT @TerryMcSweeney: #BernieInSacramento Supporters now say some were in line since 2pm. #BernieSanders comes on 8 @nbcbayarea at 11 https:/…,0,False,en,None,11,False,None,0.0000,21
4,2016-05-10 01:19:27,RT @TerryMcSweeney: #BernieInSacramento Supporters now say some were in line since 2pm. #BernieSanders comes on 8 @nbcbayarea at 11 https:/…,0,False,en,None,11,False,None,0.0000,21
5,2016-05-10 01:15:48,RT @TerryMcSweeney: #BernieInSacramento Supporters now say some were in line since 2pm. #BernieSanders comes on 8 @nbcbayarea at 11 https:/…,0,False,en,None,11,False,None,0.0000,21
6,2016-05-10 01:15:14,RT @TerryMcSweeney: #BernieInSacramento Supporters now say some were in line since 2pm. #BernieSanders comes on 8 @nbcbayarea at 11 https:/…,0,False,en,None,11,False,None,0.0000,21
7,2016-05-10 01:15:07,RT @TerryMcSweeney: #BernieInSacramento Supporters now say some were in line since 2pm. #BernieSanders comes on 8 @nbcbayarea at 11 https:/…,0,False,en,None,11,False,None,0.0000,21
8,2016-05-10 01:14:49,RT @TerryMcSweeney: #BernieInSacramento Supporters now say some were in line since 2pm. #BernieSanders comes on 8 @nbcbayarea at 11 https:/…,0,False,en,None,11,False,None,0.0000,21
9,2016-05-10 01:09:58,RT @TerryMcSweeney: #BernieInSacramento Supporters now say some were in line since 2pm. #BernieSanders comes on 8 @nbcbayarea at 11 https:/…,0,False,en,None,11,False,None,0.0000,21


In [29]:
type(doc_matrix)

scipy.sparse.csr.csr_matrix

In [30]:
# use countvectorizer to create a document-term matrix and concatinate it to your main dataframe
#(hint you should now have thousands of columns)

vect = CountVectorizer(ngram_range = (1,2), stop_words='english', min_df =2)
doc_matrix = vect.fit_transform(df.text)

In [31]:
#init_features = sp.sparse.csr_matrix(df.drop('text', axis=1).astype(float))


TypeError: cannot astype a datetimelike from [datetime64[ns]] to [float64]

In [ ]:
# 6 Use kmean to create clusters for the dataframe

In [ ]:
# 7 Explore the clusters
# What is interesting? Surprising? Can you draw any inferences about each cluster?
# Write your answer to these questions in markdown

In [ ]:
# 8 Create a column called 'democrat' and assign it to the corresponding candidate name for all the records
# 1 for bernie and hilary and 0 for donald trump

In [ ]:
# 9 Estimate classification models using knn and logistic regession.  Dont forget to use cross validation 
    # (do not use cross_val_score because we will want a confusion matrix).
# Use a three class response variable for knn.  For logistic regression predict democrat or republic?
#Assess the pros and cons of both approaches
#Extra Credit: Estimate multinomial regression model using the three class response variable

In [ ]:
# 10 Initialize a classification model (for both knn and logistic)

In [ ]:
# 11 Train your classification model  (for both knn and logistic)

In [ ]:
# 12 Predict on your test features and targets (for both knn and logistic)

In [ ]:
# 13 output your accuracy (for both knn and logistic)

In [ ]:
# 14 Create a confusion matrix for your predictions (for both knn and logistic)

In [ ]:
# 15 Who is the easiest candidate to predict? Why do you think? Who is the hardest? Why do you think?
# Write your answers in markdown (for both knn and logistic)

In [ ]:
# 16 What are some extra features (data points) you would like to add to this data set. Explain in markdown.
# (hint maybe something like a boolean, "has_hashtag")

In [ ]:
# 17 Try adding extra features to improve the quality of your predictions (for both knn and logistic)